In [5]:
import os
import pysam
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio import Align
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [6]:
def read_list(filename):
    reads=[]
    des = []
    bamfile = pysam.AlignmentFile(filename, "rb")
    for b_read in bamfile:
        reads.append(b_read.query_sequence)
        des.append(b_read.query_name)
    return reads, des

In [3]:
def sort_reads(bamfile, folder_name):
    barcodes_R=["CCCTATGACA", "TAATGGCAAG", "AACAAGGCGT", "GTATGTAGAA", "TTCTATGGGG", "CCTCGCAACC", "TGGATGCTTA", "AGAGTGCGGC"]
    barcodes_F=["aatcccactac", "tgaactgagcg", "TATCTGACCTT", "ATATGAGACG", "CGCTCATTAG", "TAATCTCGTC", "GCGCGATTTT", "AGAGCACTAG", "TGCCTTGATC", "CTACTCAGTC", "TCGTCTGACT", "GAACATACGG"]
    a, a_des = read_list(f'{folder_name}/{bamfile}')
    
    index = []
    act_barcode=[]
    for i in range(len(a)):
        barcodes = []
        bar = []
        for barcode in barcodes_R:
            rev = Seq(barcode).reverse_complement()
            if barcode.upper() in a[i] or str(rev) in a[i]:
                barcodes.append(barcodes_R.index(barcode) + 1)
                bar.append("R")
        for barcode in barcodes_F:
            rev = Seq(barcode).reverse_complement()
            if barcode.upper() in a[i] or str(rev).upper() in a[i]:   
                barcodes.append(barcodes_F.index(barcode) + 1)
                bar.append("F")
        bar = "_".join(bar)
        if bar == "R_F" and len(barcodes) == 2:
            act_bar = f'{barcodes[0]}_{barcodes[1]}'               
            with open(f'{folder_name}/{act_bar}.fasta', 'a') as f:
                f.write(f">{bar}\n")
                f.write(f'{a[i]}\n')

In [7]:
sort_reads("barcode22-extracted-sorted.bam", "Georgii/georgii_231018_1/")

[W::hts_idx_load3] The index file is older than the data file: Georgii/georgii_231018_1//barcode22-extracted-sorted.bam.bai


In [25]:
seqs = [] 
folder = "Georgii/georgii_231018_2"
subfolder = folder.split("/")[-1]
for file in os.listdir(f'{folder}/consensus'):
    if file.endswith("fasta"):
        filename=file.split(".")[0]
        for i in SeqIO.parse(f'{folder}/consensus/{file}', 'fasta'):
             dna_read = SeqRecord(i.seq, id=f'{filename}')
    seqs.append(dna_read)
SeqIO.write(seqs, f"{folder}/final/{subfolder}_consensus_96_seqs_dna.fasta", "fasta")

96